# 4. Explaining and Resolving Unsatisfiable Problems

In the previous notebook, we saw how real-world disruptions can make a Workforce Allocation Problem unsatisfiable (UNSAT). The solver correctly reported that no solution exists, but this isn't very helpful for a planner who needs to find a workable compromise.

In this final notebook, we'll address the crucial question: **Why is the problem infeasible?** We'll use a powerful technique called **Minimal Unsatisfiable Subset (MUS)** analysis to pinpoint the exact set of conflicting constraints. Then, we'll build an **interactive tool** that allows a user to diagnose the conflict, see a potential "best-effort" repaired schedule, and decide which constraints to relax to find a feasible solution.

### Prerequisites

This notebook requires the `discrete-optimization` library with its `cpmpy` and `ortools` dependencies.

In [ ]:
# On Colab: install the library
import sys

ON_COLAB = "google.colab" in sys.modules
if ON_COLAB:
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install discrete-optimization

### Imports

In [ ]:
from datetime import datetime

from utils_interact_solve import InteractSolve

from discrete_optimization.datasets import fetch_data_from_cp25
from discrete_optimization.generic_tools.cpmpy_tools import CpmpyExplainUnsatMethod
from discrete_optimization.generic_tools.do_solver import StatusSolver

# Import problem, parser, and additional constraints
from discrete_optimization.workforce.allocation.parser import (
    build_allocation_problem_from_scheduling,
)
from discrete_optimization.workforce.allocation.solvers.cpsat import (
    CpsatTeamAllocationSolver,
)

# Import utilities for visualization and problem manipulation
from discrete_optimization.workforce.generators.resource_scenario import (
    ParamsRandomness,
    generate_allocation_disruption,
)
from discrete_optimization.workforce.scheduling.parser import (
    get_data_available,
    parse_json_to_problem,
)
from discrete_optimization.workforce.scheduling.utils import (
    alloc_solution_to_alloc_sched_solution,
    build_scheduling_problem_from_allocation,
    plotly_schedule_comparison,
)

### Create the Unsatisfiable Problem via Disruption

We will now create an UNSAT problem by first solving the original problem to get a baseline, and then applying a significant disruption to it. This mirrors a realistic workflow where an existing plan is broken by new events.

In [ ]:
# Load the original problem
files = get_data_available()
if not files:
    fetch_data_from_cp25()
    files = get_data_available()
file_path = files[1]
scheduling_problem = parse_json_to_problem(file_path)
original_problem = build_allocation_problem_from_scheduling(problem=scheduling_problem)

# 1. Solve the original problem to get a baseline solution
solver_cpsat = CpsatTeamAllocationSolver(original_problem)
solver_cpsat.init_model()
sol_cpsat = solver_cpsat.solve(time_limit=30).get_best_solution()

# 2. Generate a disruption scenario based on this solution
disruption_scenario = generate_allocation_disruption(
    original_allocation_problem=original_problem,
    original_solution=sol_cpsat,
    params_randomness=ParamsRandomness(
        upper_nb_disruption=20,
        lower_nb_teams=1,
        upper_nb_teams=2,
        duration_discrete_distribution=([100], [1]),
    ),
)
problem_unsat = disruption_scenario["new_allocation_problem"]
print("Disrupted problem created.")

# 3. Create scheduling-level objects for visualization
sched_problem = build_scheduling_problem_from_allocation(
    problem=problem_unsat,
    horizon_start_shift=datetime(day=1, month=7, year=2025).timestamp(),
)
base_sched_solution = alloc_solution_to_alloc_sched_solution(
    problem=sched_problem, solution=sol_cpsat
)

## Interactive Conflict Resolution

Now, we will replicate the logic from the `interactive_solving_with_interact_obj()` function. We'll define a helper class, `InteractSolve`, to manage the solver state and constraints, making the interactive loop cleaner.

In [ ]:
# At each step, if the problem is unsatisfiable, we will compute an MUS,
# display the conflicting constraints to the user, and ask which one to relax.
# This process continues until a feasible solution is found.
interact_solve = InteractSolve(problem=problem_unsat)
solver = interact_solve.solver

done = False
removed_constraints = []
final_solution = None

print("--- Starting Interactive Solver ---")
while not done:
    print("\nAttempting to solve...")
    status, result_store = interact_solve.solve_current_problem()
    print(f"Solver status: {status}")

    if status == StatusSolver.UNSATISFIABLE:
        print(
            "\nProblem is UNSATISFIABLE. Finding a Minimal Unsatisfiable Subset (MUS)..."
        )
        mus = solver.explain_unsat_meta(
            soft=interact_solve.current_soft,
            hard=interact_solve.current_hard,
            cpmpy_method=CpmpyExplainUnsatMethod.mus,
            solver="exact",
        )

        print("\n--- CONFLICT FOUND ---")
        print("The following constraints are in conflict:")
        for i, meta_constraint in enumerate(mus):
            print(
                f"  [{i}]: Type='{meta_constraint.metadata['type']}', Details={meta_constraint.metadata}"
            )

        # Solve a relaxed problem to get a visual proposal for a fix
        print("\nSolving a relaxed problem to visualize a potential fix...")
        relaxed_alloc_sol = interact_solve.solve_relaxed_problem(
            base_solution=disruption_scenario["new_solution"]
        )
        repaired_sched_sol = alloc_solution_to_alloc_sched_solution(
            sched_problem, relaxed_alloc_sol
        )

        # Visualize the conflict (red) and dropped tasks (orange)
        tasks_in_conflict = []
        for mc in mus:
            if mc.metadata["type"] == "allocated_task":
                tasks_in_conflict.append(mc.metadata["task_index"])

        color_map = {task: "red" for task in tasks_in_conflict}
        for task in interact_solve.dropped_tasks:
            color_map[task] = "orange"

        fig = plotly_schedule_comparison(
            base_solution=base_sched_solution,
            updated_solution=repaired_sched_sol,
            problem=sched_problem,
            use_color_map_per_task=True,
            plot_team_breaks=True,
            color_map_per_task=color_map,
            display=False,
            title="Conflict Analysis: Original Plan (top) vs. Repaired Plan (bottom)",
        )
        print(
            "Visualizing conflict: Red=Involved in MUS, Orange=Dropped in repaired plan"
        )
        fig.show()

        # Interactive part
        try:
            choice = int(input("Enter the index of the constraint to relax/remove: "))
            if not 0 <= choice < len(mus):
                raise ValueError

            constraint_to_remove = mus[choice]
            removed_constraints.append(constraint_to_remove)
            interact_solve.drop_constraints([constraint_to_remove])
            print(f"--> Relaxed constraint: {constraint_to_remove.metadata}")

        except (ValueError, IndexError):
            print("Invalid input. Exiting.")
            done = True

    elif status in {StatusSolver.OPTIMAL, StatusSolver.SATISFIED}:
        final_solution = result_store.get_best_solution()
        done = True
    else:
        print("Solver returned an unexpected status or timed out. Exiting.")
        done = True

if final_solution:
    print("\n--- FEASIBLE SOLUTION FOUND! ---")
    print(f"Solution uses {int(final_solution.kpis['nb_teams'])} teams.")

    print("\nRelaxed constraints to achieve feasibility:")
    if not removed_constraints:
        print("  None.")
    for c in removed_constraints:
        print(f"  - Type: {c.metadata['type']}, Details: {c.metadata}")
        if c.metadata["type"] == "allocated_task":
            task_name = problem_unsat.index_to_activities_name[c.metadata["task_index"]]
            print(f"    (This means task '{task_name}' was dropped from the schedule)")
    fig = plotly_schedule_comparison(
        base_solution=base_sched_solution,
        updated_solution=alloc_solution_to_alloc_sched_solution(
            sched_problem, final_solution
        ),
        problem=sched_problem,
        plot_team_breaks=True,
        display=False,
        title="Conflict Analysis: Original Plan (top) vs. Repaired Plan (bottom)",
    )
    fig.show()

## Conclusion

Thank you on completing this notebook series! 

On those 4 first notebooks We have came from the basic definition of the Workforce Allocation Problem to advanced, interactive methods for handling real-world complexities. You have learned how to:

1.  **Define and Visualize** a complex assignment problem.
2.  **Solve it to Optimality** using Constraint Programming.
3.  **Model Disruptions** that can make a plan infeasible.
4.  **Diagnose and Resolve** infeasibility by finding the core conflicts (MUS) and interactively relaxing constraints.
Soon, to come : using minimal correction subset and a rescheduling approach to resolve infeasible disrupted scenarios.

